***
## Parte III
O objetivo desta parte do trabalho é analisar o comportamento dos índices das tabelas do SGBD através
do exame e análise das tabelas de estatísticas para consultas SQL sobre as tabelas “movies”, “actors”,
“casting”, e sobre uma tabela criada com dados aleatórios. Esta tarefa deverá ser executada somente com
o PostgreSQL.

- _Alice Teles Lucena_
- _Igor de Souza Lima_
- _Nathália Rodrigues Machado dos Santos_

In [1]:
# Fazendo a conexão com o banco de dados
import psycopg2

conn = psycopg2.connect( 
    host="localhost",
    port="5432",
    database="tpchdb",
    user="tpch",
    password="test123"
)

cursor = conn.cursor()

print("Conexão estabelecida com o Banco de Dados !!!")

Conexão estabelecida com o Banco de Dados !!!


--------------------------------------------
### Tarefa 10 – Preparação de Tabela Exemplo
Criar uma tabela com uma chave simples e alguns dados de exemplo. Cada valor de chave é um número
incremental e está associado a com valores que variam de 0 até 10:

> `DROP TABLE IF EXISTS t;`
> 
> `CREATE TABLE t (k serial PRIMARY KEY, v integer);`
> 
> `INSERT INTO t(v)`
> 
> `SELECT trunc(random() * 10) FROM generate_series(1,100000);`

__Entrega__: Imprimir os valores das 10 primeiras tuplas da tabela, ordenando por k.

<span style="color:orange">__Resposta:__</span>

In [2]:
# Criando a tabela
cursor.execute("DROP TABLE IF EXISTS t; CREATE TABLE t (k serial PRIMARY KEY, v integer); INSERT INTO t(v) SELECT trunc(random() * 10) FROM generate_series(1,100000);")

In [3]:
# Executando a query para imprimir os valores das 10 primeiras tuplas
cursor.execute("SELECT * FROM t LIMIT 10 OFFSET 0;")

rows = cursor.fetchall()
for row in rows:
    print(row)

(1, 3)
(2, 1)
(3, 3)
(4, 7)
(5, 0)
(6, 5)
(7, 5)
(8, 6)
(9, 8)
(10, 7)


--------------------------------------------
### Tarefa 11 – Páginas criadas
Verifique quantas páginas com blocos foram criadas para a tabela da Tarefa 11.
>`SELECT relname, relpages, reltuples FROM pg_class WHERE relname='t';`

__Entrega:__ Imprimir o resultado do comando SQL.

<span style="color:orange">__Resposta:__</span>

In [4]:
# Executando o comando SQL
cursor.execute("""
    SELECT ceil(pg_total_relation_size('t') / current_setting('block_size')::numeric)
    FROM pg_class
    WHERE relname = 't'
""")

result = cursor.fetchone()
print("Total de páginas:", result[0])

Total de páginas: 722


--------------------------------------------
### Tarefa 12 – Blocos
Verifique quantos blocos foram efetivamente usados numa consulta

>`SELECT pg_sleep(1);`
>
>`\pset x on`
>
>`SELECT * FROM pg_stats WHERE relname='t';`
>
>`SELECT pg_stat_reset();`
>
>`\pset x off`

__Observação:__ Em algumas versões do PostgreSQL, o atributo é chamado de tablename em vez de
relname.

__Entrega:__ Imprimir o resultado do comando SQL.

<span style="color:orange">__Resposta:__</span>

In [5]:
cursor.execute("EXPLAIN (ANALYZE, BUFFERS) SELECT * FROM t ORDER BY k LIMIT 10")

result = cursor.fetchall()

for row in result:
    if "Buffers:" in row[0]:
        blocks_used = int(row[0].split("=")[1].split()[0])
        print("Total de blocos na consulta: ", blocks_used)
        break

Total de blocos na consulta:  3


--------------------------------------------
### Tarefa 13 – Índice
Crie um índice para o atributo ‘v’ e realize consultas e criação de índice

a) Qual o tempo gasto para realizar uma consulta para um valor (tendo a tabela 100000 tuplas)?

<span style="color:orange">__Resposta Letra A:__</span>

In [6]:
import time

# Criando um índice para o atributo 'v'
cursor.execute("CREATE INDEX idx_v ON t(v);")

In [7]:
# Realizando uma consulta para um valor específico (100)
start_time = time.time()
cursor.execute("SELECT * FROM t WHERE v = 100;")
end_time = time.time()
execution_time = end_time - start_time
print("Tempo gasto para realizar a consulta: ", execution_time, " segundos")

Tempo gasto para realizar a consulta:  0.0008096694946289062  segundos


b) Qual o tempo gasto para recriar um índice para o atributo ‘v’?

<span style="color:orange">__Resposta Letra B:__</span>

In [8]:
# Removendo o índice existente
cursor.execute("DROP INDEX idx_v;")

# Recriando o índice para o atributo 'v'
start_time = time.time()
cursor.execute("CREATE INDEX idx_v ON t(v);")
end_time = time.time()
execution_time = end_time - start_time
print("Tempo gasto para recriar o índice: ", execution_time, " segundos")

Tempo gasto para recriar o índice:  0.5513639450073242  segundos


Remova a tabela ‘t’ e crie novamente com 1.0000.000 de tuplas

c) Qual o tempo gasto para realizar uma consulta para um valor específico?

<span style="color:orange">__Resposta Letra C:__</span>

In [9]:
# Removendo a tabela existente
cursor.execute("DROP TABLE IF EXISTS t;")

# Criando novamente a tabela com 1.000.000 de tuplas
cursor.execute("CREATE TABLE t (k serial PRIMARY KEY, v integer); INSERT INTO t(v) SELECT trunc(random() * 10) FROM generate_series(1,1000000);")

# Realizando uma consulta para um valor específico (100)
start_time = time.time()
cursor.execute("SELECT * FROM t WHERE v = 100;")
end_time = time.time()
execution_time = end_time - start_time
print("Tempo gasto para realizar a consulta: ", execution_time, " segundos")

Tempo gasto para realizar a consulta:  0.03661775588989258  segundos


d) Qual o tempo gasto para recriar um índice para o atributo ‘v’?

<span style="color:orange">__Resposta Letra D:__</span>

In [10]:
# Verificando a existência do índice
cursor.execute("""
    SELECT EXISTS (
        SELECT 1
        FROM pg_indexes
        WHERE indexname = 'idx_v');
""")

index_exists = cursor.fetchone()[0]

if index_exists:
    # Removendo o índice existente
    cursor.execute("DROP INDEX idx_v;")

# Recriando o índice para o atributo 'v'
start_time = time.time()
cursor.execute("CREATE INDEX idx_v ON t(v);")
end_time = time.time()
execution_time = end_time - start_time
print("Tempo gasto para recriar o índice: ", execution_time, " segundos")

Tempo gasto para recriar o índice:  0.9239134788513184  segundos


--------------------------------------------
### Tarefa 14 - Fill factor
Quando se cria um novo índice, nem toda entrada no bloco do índice é usada. Um espaço livre é deixado,
conforme o parâmetro fillfactor.

Crie novos índices usando fillfactor = 60, 80, 90 e 100. Analise o desempenho de suas consultas usando as
mesmas condições da Tarefa 14.

>`ALTER TABLE foo SET ( fillfactor = 50);`
>
> `VACUUM FULL foo;`

__Entrega:__ Relatório com o resultado das perguntas

<span style="color:orange">__Resposta:__</span>

In [11]:
# FILLFACTOR 60
cursor.execute("CREATE INDEX idx_60 ON t(v) WITH (fillfactor = 60);")

# Realizando uma consulta para um valor específico (100)
start_time = time.time()
cursor.execute("SELECT * FROM t WHERE v = 100;")
end_time = time.time()
execution_time = end_time - start_time
print("Tempo gasto para realizar a consulta: ", execution_time, " segundos")

Tempo gasto para realizar a consulta:  0.001329183578491211  segundos


In [12]:
# FILLFACTOR 80
cursor.execute("CREATE INDEX idx_80 ON t(v) WITH (fillfactor = 80);")

# Realizando uma consulta para um valor específico (100)
start_time = time.time()
cursor.execute("SELECT * FROM t WHERE v = 100;")
end_time = time.time()
execution_time = end_time - start_time
print("Tempo gasto para realizar a consulta: ", execution_time, " segundos")

Tempo gasto para realizar a consulta:  0.0013377666473388672  segundos


In [13]:
# FILLFACTOR 90
cursor.execute("CREATE INDEX idx_90 ON t(v) WITH (fillfactor = 90);")

# Realizando uma consulta para um valor específico (100)
start_time = time.time()
cursor.execute("SELECT * FROM t WHERE v = 100;")
end_time = time.time()
execution_time = end_time - start_time
print("Tempo gasto para realizar a consulta: ", execution_time, " segundos")

Tempo gasto para realizar a consulta:  0.0012090206146240234  segundos


In [14]:
# FILLFACTOR 100
cursor.execute("CREATE INDEX idx_100 ON t(v) WITH (fillfactor = 100);")

# Realizando uma consulta para um valor específico (100)
start_time = time.time()
cursor.execute("SELECT * FROM t WHERE v = 100;")
end_time = time.time()
execution_time = end_time - start_time
print("Tempo gasto para realizar a consulta: ", execution_time, " segundos")

Tempo gasto para realizar a consulta:  0.0014634132385253906  segundos


--------------------------------------------
### Tarefa 15 – Usando índice com múltiplas colunas
- Use as tabelas movies, actor e casting e realize a criação de índice que utilizem 2, 3 e 4 colunas.
- Use a tabela com 1000 tuplas e relate o desempenho de suas consultas.
- Use a tabela com mais de 100.000 tuplas e relate o desempenho de suas consultas.

__Entrega:__ Relatório com o resultado do desempenho

<span style="color:red">__QUESTÃO ANULADA__</span>

--------------------------------------------
### Tarefa 16 - Utilize índices com ordem DESC
Repita os testes das Tarefas 11,12 e 13 usando índices descendentes. Avalie e registre na ficha

> `CREATE INDEX i ON t(v DESC NULLS FIRST);`

__Entrega:__ Relatório com o resultado da avaliação

<span style="color:orange">__Resposta:__</span>

In [26]:
# REFAZENDO TAREFA 11 - ÍNDICES DECRESCENTES
# -------------------------------------------------
# Executando o comando SQL

cursor.execute("CREATE INDEX idx_v11 ON t(v DESC NULLS FIRST);")
cursor.execute("""
    SELECT ceil(pg_total_relation_size('t') / current_setting('block_size')::numeric)
    FROM pg_class
    WHERE relname = 't'
""")

result = cursor.fetchone()
print("Total de páginas:", result[0])

Total de páginas: 809


In [27]:
# REFAZENDO TAREFA 12 - ÍNDICES DECRESCENTES
# -------------------------------------------------
cursor.execute("EXPLAIN (ANALYZE, BUFFERS) SELECT * FROM t ORDER BY k LIMIT 10")

result = cursor.fetchall()

for row in result:
    if "Buffers:" in row[0]:
        blocks_used = int(row[0].split("=")[1].split()[0])
        print("Total de blocos na consulta: ", blocks_used)
        break

Total de blocos na consulta:  3


In [28]:
# REFAZENDO TAREFA 13 - ÍNDICES DECRESCENTES
# -------------------------------------------------

# Criando um índice para o atributo 'v'
cursor.execute("CREATE INDEX idx_v13 ON t(v DESC NULLS FIRST);")

# Realizando uma consulta para um valor específico (100)
start_time = time.time()
cursor.execute("SELECT * FROM t WHERE v = 100;")
end_time = time.time()
execution_time = end_time - start_time
print("a) Tempo gasto para realizar a consulta: ", execution_time, " segundos")

# -------------------------------------------------
# Removendo o índice existente
cursor.execute("DROP INDEX idx_v13;")

# Recriando o índice para o atributo 'v'
start_time = time.time()
cursor.execute("CREATE INDEX idx_v13 ON t(v DESC NULLS FIRST);")
end_time = time.time()
execution_time = end_time - start_time
print("b) Tempo gasto para recriar o índice: ", execution_time, " segundos")

# -------------------------------------------------
# Removendo a tabela existente
cursor.execute("DROP TABLE IF EXISTS t;")

# Criando novamente a tabela com 1.000.000 de tuplas
cursor.execute("CREATE TABLE t (k serial PRIMARY KEY, v integer); INSERT INTO t(v) SELECT trunc(random() * 10) FROM generate_series(1,1000000);")

# Realizando uma consulta para um valor específico (100)
start_time = time.time()
cursor.execute("SELECT * FROM t WHERE v = 100;")
end_time = time.time()
execution_time = end_time - start_time
print("c) Tempo gasto para realizar a consulta: ", execution_time, " segundos")

# -------------------------------------------------
# Verificando a existência do índice
cursor.execute("""
    SELECT EXISTS (
        SELECT 1
        FROM pg_indexes
        WHERE indexname = 'idx_v13');
""")

index_exists = cursor.fetchone()[0]

if index_exists:
    # Removendo o índice existente
    cursor.execute("DROP INDEX idx_v13;")

# Recriando o índice para o atributo 'v'
start_time = time.time()
cursor.execute("CREATE INDEX idx_v13 ON t(v DESC NULLS FIRST);")
end_time = time.time()
execution_time = end_time - start_time
print("d) Tempo gasto para recriar o índice: ", execution_time, " segundos")

a) Tempo gasto para realizar a consulta:  0.0015437602996826172  segundos
b) Tempo gasto para recriar o índice:  0.1456279754638672  segundos
c) Tempo gasto para realizar a consulta:  0.025614500045776367  segundos
d) Tempo gasto para recriar o índice:  0.8664169311523438  segundos
